<a href="https://colab.research.google.com/github/d3nnyyy/AI-Exploration/blob/master/DeepLearning/StudentsRatings/students_ratings_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1: Importing Libraries and Loading Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the data
data = pd.read_csv('rating.csv')

# Extract features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Data preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


FileNotFoundError: ignored

# Section 2: Define the Neural Network Model

In [ ]:
# Define the neural network model
class ImportanceModel(nn.Module):
    def __init__(self, input_size):
        super(ImportanceModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
input_size = X_train.shape[1]
model = ImportanceModel(input_size)

# Section 3: Define Loss Function and Optimizer

In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Section 4: Convert Data to PyTorch Tensors

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)

X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

# Section 5: Training the Model

In [ ]:
# Training
epochs = 500

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


# Section 6: Model Evaluation

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Mean Squared Error on Test Set: {test_loss.item():.4f}')

# Section 7: Get Importance Scores

In [ ]:
# Get importance scores
importance_scores = model.fc1.weight.data.numpy()[0]
print('Importance Scores:', importance_scores)

# Section 8: Make Predictions on Random Students

In [ ]:
# Make predictions on random students
import numpy as np

random_students = np.array([
    [90, 88, 82, 95, 96, 88, 97],
    [60, 71, 54, 92, 98, 99, 100],
    [71, 90, 71, 91, 93, 72, 90],
    [71, 74, 54, 94, 87, 95, 92],
    [88, 82, 76, 83, 96, 74, 89],
    [71, 79, 78, 89, 88, 98, 97],
    [67, 56, 64, 93, 82, 69, 79],
    [71, 88, 72, 100, 100, 71, 99],
    [60, 60, 50, 100, 88, 51, 64],
    [92, 80, 88, 100, 100, 90, 85],
    [83, 58, 54, 100, 100, 100, 93],
    [95, 88, 65, 74, 98, 53, 100],
    [50, 53, 50, 50, 50, 50, 73],
    [56, 76, 50, 50, 50, 51, 69],
    [92, 74, 50, 97, 93, 71, 88],
    [78, 64, 52, 84, 94, 71, 88],
    [74, 71, 50, 88, 98, 71, 79],
    [76, 75, 50, 73, 85, 50, 94],
    [51, 66, 62, 67, 99, 60, 57],
    [50, 50, 50, 50, 50, 50, 50],
    [82, 66, 51, 88, 95, 71, 90],
    [54, 71, 73, 76, 83, 52, 84],
    [71, 88, 73, 94, 77, 73, 77],
    [75, 89, 88, 75, 95, 51, 89],
    [62, 59, 50, 65, 73, 51, 70],
    [83, 81, 61, 53, 95, 100, 88],
    [85, 84, 91, 94, 100, 88, 96]
]
, dtype=np.float32)

# Standardize the input using the same scaler used during training
random_students_scaled = scaler.transform(random_students)

# Convert to PyTorch tensor
random_students_tensor = torch.FloatTensor(random_students_scaled)


# Section 9: Calculate Final Marks and Rank Students

In [ ]:
# Predict final marks
model.eval()
with torch.no_grad():
    predicted_ratings = model(random_students_tensor)

# Calculate final marks using importance scores
final_marks = torch.sum(predicted_ratings * importance_scores, dim=1)  # Assuming a linear combination

# Load real names
names_df = pd.read_csv('rating-names.csv', encoding='utf-8')
real_names = names_df['student'].tolist()

# Rank students
rankings = torch.argsort(final_marks, descending=True)

# Print rankings with real names
for i, rank in enumerate(rankings):
    student_idx = rank.item()
    student_name = real_names[student_idx]
    print(f"Rank {i + 1}: {student_name}")
